In [ ]:
import requests
import json
import pprint
import pandas as pd
import numpy as np
#!pip install xmltodict
import xmltodict
from xml.etree import ElementTree
#!pip install pymed
from pymed import PubMed
#!pip install Bio
from Bio import Entrez
#!pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

In [ ]:
#https://stackoverflow.com/questions/57053378/query-pubmed-with-python-how-to-get-all-article-details-from-query-to-pandas-d
pubmed = PubMed(tool="PubMedSearcher", email="martina.fonseca@nhsx.nhs.uk")
results = pubmed.query('nhsx[affiliation]', max_results=500) # number might need to be updated in future, for now low
articleList = []
articleInfo = []

for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    articleDict = article.toDict() # convert to dictionary
    articleList.append(articleDict)

# Generate list of dict records which will hold all article details that could be fetch from PUBMED API
for article in articleList:
#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
    pubmedId = article['pubmed_id'].partition('\n')[0] # keep only pubmed id
    # Append article info to dictionary #
    articleInfo.append({u'pubmed_id':pubmedId,
                       u'title':article['title'],
                       u'keywords':article['keywords'],
                       u'journal':article['journal'],
                       u'abstract':article['abstract'],
                       u'conclusions':article['conclusions'],
                       u'methods':article['methods'],
                       u'results': article['results'],
                       u'copyrights':article['copyrights'],
                       u'doi':article['doi'],
                       u'publication_date':article['publication_date'], 
                       u'authors':article['authors']})

# Convert list of dictionaries to pandas DataFrame
articlesPD = pd.DataFrame.from_dict(articleInfo)
#export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

#Print first 10 rows of dataframe
articlesPD.head(10)
#print(articlesPD['title'])
#print(articlesPD['keywords'])
#print(articleList)

#### Unravel author list, retain initial + surname

In [ ]:
i=0
Authors_np=np.array([])
for element in articlesPD["authors"]:
    mystring= ""
    for author in element:
        if author['affiliation'].lower().find("nhsx")== -1:
            mystring = mystring + author['lastname'] + " " + author['initials'] + ", "
        else:
            mystring = mystring + "<b>" + author['lastname'] + " " + author['initials'] + "</b>, "
     
    Authors_np=np.append(Authors_np,mystring[:-2])
    #articlesPD["Authors"][i]=mystring[:-1]
    i=i+1

In [ ]:
articlesPD["Authors"]=Authors_np

#### Obtain number of forward cites (only from Pubmed articles cited in PMC - current API that is available)

In [ ]:
#https://gist.github.com/mcfrank/c1ec74df1427278cbe53
Entrez.email = "martina.fonseca@nhsx.nhs.uk"

In [ ]:
def get_onwardlinks_id(pmid):
  link_list = []
  links = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pmc_refs")	
  record = Entrez.read(links)
  if record[0]['LinkSetDb']!=[]:
    records = record[0][u'LinkSetDb'][0][u'Link']
    for link in records:
      link_list.append(link[u'Id'])
  return link_list

In [ ]:
#https://stackoverflow.com/questions/26483254/python-pandas-insert-list-into-a-cell
i=0
#citations_np=np.array([])
articlesPD['citations_list']=np.nan
articlesPD['citations_list']=articlesPD['citations_list'].astype(object)
for element in articlesPD["pubmed_id"]:
    print(element)
    citations_this = get_onwardlinks_id(element)
    articlesPD.at[i,'citations_list']=citations_this
    i=i+1
     
    #citations_np=np.append(citations_np,citations_this,axis=1)
#articlesPD["List_citations"]=citations_np

In [ ]:
len(citations_this)

In [ ]:
articlesPD["No. PMC Citations"]=articlesPD["citations_list"].apply(lambda x: len(x))

In [ ]:
articlesPD.head()

In [ ]:
articlesPD["DOI"] = "<a href='" + "http://doi.org/" +articlesPD["doi"] + "'>" + articlesPD["doi"] + "</a>" # create DOI column that is html link to DOI page

In [12]:
', '.join([str(x) for x in articlesPD["keywords"][2]])  # list comprehension

''

In [13]:
articlesPD["Keywords"]=articlesPD["keywords"].apply(lambda keyword_list: ', '.join([str(x) for x in keyword_list]))

In [14]:
articlesPD.head()

,pubmed_id,title,keywords,journal,abstract,conclusions,methods,results,copyrights,doi,publication_date,authors,Authors,citations_list,No. PMC Citations,DOI,Keywords
0,33094226,Digital health - a trainee's perspective.,"[QI, Trainee, digital, training]",Future healthcare journal,"As we shift towards using digital systems, and...",None,None,None,© Royal College of Physicians 2020. All rights...,10.7861/fhj.dig-2020-trai,2020-10-24,"[{'lastname': 'Maguire', 'firstname': 'James',...",<b>Maguire J</b>,[],0,<a href='http://doi.org/10.7861/fhj.dig-2020-t...,"QI, Trainee, digital, training"
1,32702587,The ethics of AI in health care: A mapping rev...,"[Artificial intelligence, Ethics, Health polic...",Social science & medicine (1982),This article presents a mapping review of the ...,None,None,None,Copyright © 2020 Elsevier Ltd. All rights rese...,10.1016/j.socscimed.2020.113172,2020-07-24,"[{'lastname': 'Morley', 'firstname': 'Jessica'...","Morley J, Machado CCV, Burr C, Cowls J, <b>Jos...","[8099995, 7955618, 7885243, 7571864]",4,<a href='http://doi.org/10.1016/j.socscimed.20...,"Artificial intelligence, Ethics, Health polici..."
2,32672131,Bringing NHS data analysis into the 21st century.,[],Journal of the Royal Society of Medicine,None,None,None,None,None,10.1177/0141076820930666,2020-07-17,"[{'lastname': 'Goldacre', 'firstname': 'Ben', ...","Goldacre B, Bardsley M, Benson T, Cheema K, Ch...",[7754812],1,<a href='http://doi.org/10.1177/01410768209306...,
3,32616598,Using imaging to combat a pandemic: rationale ...,[],The European respiratory journal,None,None,None,None,None,10.1183/13993003.01809-2020,2020-07-04,"[{'lastname': 'Jacob', 'firstname': 'Joseph', ...","Jacob J, Alexander D, Baillie JK, Berka R, Ber...","[8155634, 8085195, 7769066, 7543687]",4,<a href='http://doi.org/10.1183/13993003.01809...,
4,32010451,Supporting early clinical careers in digital h...,[],Digital health,None,None,None,None,None,10.1177/2055207619899798,2020-02-06,"[{'lastname': 'Robbins', 'firstname': 'Tim', '...","Robbins T, Zucker K, Abdulhussein H, Chaplin V...",[],0,<a href='http://doi.org/10.1177/20552076198997...,


In [15]:
articlesPD["DOIbullets"] = "<a href='" + "http://doi.org/" +articlesPD["doi"] + "'>" + articlesPD["title"] + "</a>" # create DOI column that is html link to DOI page

In [18]:
# cs branch - adapted
def ulify(elements):
    string = "<ul>"
    string += "".join(["<li>" + str(s) + "</li>" for s in elements])
    string += "</ul>"
    return string

In [19]:
html_list = ulify(articlesPD["DOIbullets"])

In [20]:
with open("_includes/publications.html", "w") as file:
    file.write(html_list)

In [21]:
articlesPD_sel = articlesPD[["publication_date","DOIbullets","journal","Authors","No. PMC Citations","Keywords"]].copy()

In [22]:
articlesPD_sel=articlesPD_sel.rename(columns={"DOIbullets": "Publication Title", "journal": "Journal","publication_date":"Publication Date"})

In [ ]:
articlesPD_sel

In [ ]:
# new style table
aggregate_latest_html = articlesPD_sel.to_html(
    index=False, render_links=True, escape=False
)
aggregate_latest_html = aggregate_latest_html.replace(
    "dataframe", "nhsuk-table__panel-with-heading-tab"
)
aggregate_latest_html = aggregate_latest_html.replace('border="1"', "")
with open("_includes/NHSUK_table.html", "w") as file:
    file.write(aggregate_latest_html)

In [23]:
# Create output table
#layout = go.Layout( autosize=True, **margin={'l': 0, 'r': 0, 't': 20, 'b': 0}**)
fig = go.Figure()
fig.add_trace(
    go.Table(
        columnorder = [1,2,3,4,5,6,7],
        columnwidth = [80,160,80,160,80,80,80],
        header=dict(
            values=["<b>" + c + "<b>" for c in articlesPD_sel.columns],
            fill_color="rgba(240, 244, 245, 1)",
            align="left",
        ),
        cells=dict(
            values=articlesPD_sel.T.values.tolist(), # dataframe > transpose > list
            fill_color="rgba(240, 244, 245, 1)",
            align="left",
        ),
    )
)

# Asthetics of the table
fig.update_layout(
    {
        "plot_bgcolor": "rgba(240, 244, 245, 1)",
        "paper_bgcolor": "rgba(240, 244, 245, 1)",
    },
    autosize=True,
    height=600,
    width=1300
)


# Write out to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_table = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/plotly_table_nhsx.html", "w") as file:
    file.write(plotly_table)

In [24]:
# Grab timestamp
data_updated = datetime.now().strftime("%d/%m/%Y")

# Write out to file (.html)
html_str = (
    '<p><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16"><path fill-rule="evenodd" d="M1.5 8a6.5 6.5 0 1113 0 6.5 6.5 0 01-13 0zM8 0a8 8 0 100 16A8 8 0 008 0zm.5 4.75a.75.75 0 00-1.5 0v3.5a.75.75 0 00.471.696l2.5 1a.75.75 0 00.557-1.392L8.5 7.742V4.75z"></path></svg> Latest Data: '
    + data_updated
    + "</p>"
)
with open("_includes/update.html", "w") as file:
    file.write(html_str)

#### Fetching also yearly publications with NHS[affiliation] authors (time-series)

In [25]:
def get_links_term(term,retmax=10000):
	links = Entrez.esearch(db="pubmed",retmax=retmax, term=term)	
	record = Entrez.read(links)
	#link_list = record[u'IdList']

	return record

In [26]:
df_nhs_y = pd.DataFrame({'year':np.arange(2010,2022)})

In [27]:
df_nhs_y.set_index('year',inplace=True) # reset index

In [28]:
df_nhs_y['publications_count']=np.nan # initialise

In [29]:
for i in df_nhs_y.index:
  search_string='(nhs[Affiliation]) AND (("' + str(i) + '/01/01"[Date - Publication] : "' + str(i) + '/12/31"[Date - Publication]))'
  df_nhs_y.loc[i]['publications_count']=get_links_term(search_string,retmax=10000000)['Count']
  print(search_string)

In [30]:
df_nhs_y.index
df_nhs_y['Year_datetime'] =  pd.to_datetime(df_nhs_y.index, format='%Y')

In [31]:
# Plot figure
fig = go.Figure([go.Bar(x=df_nhs_y.index, y=df_nhs_y['publications_count'])])

# Asthetics of the plot
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
    # height=1000,
    # hovermode="x",
)

# Add title and dynamic range selector to x axis
fig.update_xaxes(
    title_text="Date",
    rangeselector=dict(
        buttons=list(
            [
                #dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="5y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        )
    ),
)

# Add title to y axis
fig.update_yaxes(title_text="Count of publications")

# Write out to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_obj = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/plotly_obj.html", "w") as file:
    file.write(plotly_obj)

In [32]:
#x = pd.date_range(datetime.today(), periods=100).tolist()
#y = random.sample(range(1, 300), 100)

In [33]:
df_nhs_y['Year_datetime']=df_nhs_y.index
df_nhs_y.index

Int64Index([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
            2021],
           dtype='int64', name='year')